In [1]:
from sympy import Function, Symbol, E, Integral, diff, limit, sqrt, simplify, series, pi, oo, erf, cosh, ln
from sympy.stats import Normal, cdf

In [2]:
delta = Symbol('Delta', positive=True)
l = Symbol('l', positive=True)
tau = Symbol('tau', positive=True)
theta = Symbol('theta', positive=True)
mu = Symbol('mu')
sig = Symbol('sigma', positive=True)
f = Symbol('f', positive=True)

In [3]:
# tick width
delta

Delta

In [4]:
# liquidity deployed per unit of external virtual reserves
l

l

In [5]:
# rebalancing period
tau

tau

In [6]:
# pool fee volume per unit of external virtual reserves
theta

theta

In [7]:
# pool fee rate
f

f

In [8]:
# log-normal drift
mu

mu

In [9]:
# log-normal vol
sig

sigma

In [10]:
# log-normal drift adjusted for vol
mu_p = mu - sig**2 / 2
mu_p

mu - sigma**2/2

In [11]:
# standard normal RV
Z = Normal('Z', 0, 1)
Z

Z

In [12]:
# alias standard norm cdf: Phi(z)
Phi = cdf(Z)

In [13]:
# "normalized" delta (+)
d_plus = (delta - mu_p * tau) / (sig * sqrt(tau))
d_plus

(Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau))

In [14]:
# "normalized" delta (-)
d_minus = (-delta - mu_p * tau) / (sig * sqrt(tau))
d_minus

(-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau))

In [15]:
# term for fee revenue EV (integral unevaluated)
t = Symbol('t')
d_plus_t = (delta - mu_p * t) / (sig * sqrt(t))
d_minus_t = (-delta - mu_p * t) / (sig * sqrt(t))

integrand_fee = Phi(d_plus_t) - Phi(d_minus_t) + E**(mu * tau) * ( Phi(d_plus_t - sig * sqrt(t)) - Phi(d_minus_t - sig * sqrt(t)) )

ev_fee = (theta / (1-E**(-delta/2) + l)) * Integral(integrand_fee, (t, 0, tau))
ev_fee

theta*Integral((-erf(sqrt(2)*(-sigma*sqrt(t) + (-Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(t) + (Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2 + erf(sqrt(2)*(Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2, (t, 0, tau))/(l + 1 - exp(-Delta/2))

In [16]:
# TODO: re-evaluate integral on wolfram


In [17]:
# terms for principal before rebalancing
ev_principal_before_a = (E**(delta/2) + 1) * (E**(mu*tau) * Phi(d_minus - sig * sqrt(tau)) + 1 - Phi(d_plus))
ev_principal_before_a

(exp(Delta/2) + 1)*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)

In [18]:
ev_principal_before_b = (1/(1-E**(-delta/2))) * (2 * E**((mu-sig**2/4) * (tau/2))) * (Phi(d_plus - sig * sqrt(tau) / 2) - Phi(d_minus - sig * sqrt(tau) / 2))
ev_principal_before_b

2*(-erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(tau*(mu - sigma**2/4)/2)/(1 - exp(-Delta/2))

In [19]:
ev_principal_before_c = -(E**(-delta/2)/(1-E**(-delta/2))) * (Phi(d_plus) - Phi(d_minus) + E**(mu*tau) * (Phi(d_plus - sig * sqrt(tau)) - Phi(d_minus - sig * sqrt(tau))))
ev_principal_before_c

-((-erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2)*exp(-Delta/2)/(1 - exp(-Delta/2))

In [20]:
ev_principal_before = ev_principal_before_a + ev_principal_before_b + ev_principal_before_c

In [21]:
# terms for swap fees paid on rebalance
ev_swap_fees_a = - (f/2) * (E**(delta/2) + 1) * (E**(mu * tau) * Phi(d_minus - sig * sqrt(tau)) + 1 - Phi(d_plus))
ev_swap_fees_a

-f*(exp(Delta/2) + 1)*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)/2

In [22]:
ev_swap_fees_b = - (f/2) * (1/(E**(delta/2) - 1)) * (E**(mu * tau) * (Phi(d_plus - sig*sqrt(tau)) + Phi(d_minus - sig * sqrt(tau)) - 2 * Phi(-mu_p * tau / (sig * sqrt(tau)) - sig * sqrt(tau))) + 2*Phi(-mu_p * tau/(sig * sqrt(tau))) - Phi(d_plus) - Phi(d_minus))
ev_swap_fees_b

-f*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 - erf(sqrt(2)*(-sigma*sqrt(tau) + sqrt(tau)*(-mu + sigma**2/2)/sigma)/2))*exp(mu*tau) - erf(sqrt(2)*(-Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + erf(sqrt(2)*sqrt(tau)*(-mu + sigma**2/2)/(2*sigma)))/(2*(exp(Delta/2) - 1))

In [23]:
ev_swap_fees = ev_swap_fees_a + ev_swap_fees_b

In [24]:
# terms for slippage paid on rebalance
ev_slippage_a = -(l/4) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (E**(delta/2) + 1)**2 * (E**(-mu*tau) * (1 - Phi(d_plus + sig * sqrt(tau)/2)) + E**(mu*tau) * Phi(d_minus - 3*sig*sqrt(tau) / 2))
ev_slippage_a

-l*((1/2 - erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau) + (erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau))*(exp(Delta/2) + 1)**2*exp(tau*(mu + 3*sigma**2/4)/2)/4

In [25]:
ev_slippage_b = -(l/4) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (1/(E**(delta/2) - 1)**2) * (E**(mu * tau) * (Phi(d_plus - 3 * sig * sqrt(tau) / 2) - Phi(d_minus - 3 * sig * sqrt(tau) / 2)) + E**(-mu*tau) * (Phi(d_plus + sig * sqrt(tau) / 2) - Phi(d_minus + sig * sqrt(tau) / 2)))
ev_slippage_b

-l*((-erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(mu*tau) + (-erf(sqrt(2)*(sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau))*exp(tau*(mu + 3*sigma**2/4)/2)/(4*(exp(Delta/2) - 1)**2)

In [26]:
ev_slippage_c = (l/2) * E**((mu + 3 * sig**2/4) * (tau / 2)) * (1/(E**(delta/2) - 1)**2) * (E**(-sig**2 * tau / 2)) * (Phi(d_plus - sig * sqrt(tau)/2) - Phi(d_minus - sig * sqrt(tau)/2))
ev_slippage_c

l*(-erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-sigma**2*tau/2)*exp(tau*(mu + 3*sigma**2/4)/2)/(2*(exp(Delta/2) - 1)**2)

In [27]:
ev_slippage = ev_slippage_a + ev_slippage_b + ev_slippage_c

In [28]:
ev_principal = ev_principal_before + ev_swap_fees + ev_slippage

In [29]:
rho_bar = ev_principal

In [30]:
psi_bar = ev_fee

In [31]:
# v_bar = ev of LP value
v_bar = ev_principal + ev_fee

In [32]:
v_bar_fn = Function('V')(delta, l)
v_bar_fn

V(Delta, l)

In [33]:
rho_bar_fn = Function('rho')(delta, l)
rho_bar_fn

rho(Delta, l)

In [34]:
psi_bar_fn = Function('psi')(delta, l)
psi_bar_fn

psi(Delta, l)

### Expansion about $(0, 0)$

For first order expansion about $(\Delta, l) \to (0, 0)$, need the following terms at $(0, 0)$:

- $\partial_{\Delta} V$
- $\partial_{l} V$
- $\partial^2_{l} V$
- $\partial^2_{\Delta} V$
- $\partial_{l} \partial_{\Delta} V$

In [35]:
# TODO: use wolfram to integrate fees expression for limits

In [36]:
diff(v_bar, delta)

-f*(-sqrt(2)*exp(mu*tau)*exp(-(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))**2/2)/(2*sqrt(pi)*sigma*sqrt(tau)) - sqrt(2)*exp(-(Delta - tau*(mu - sigma**2/2))**2/(2*sigma**2*tau))/(2*sqrt(pi)*sigma*sqrt(tau)))*(exp(Delta/2) + 1)/2 - f*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau) - erf(sqrt(2)*(Delta - tau*(mu - sigma**2/2))/(2*sigma*sqrt(tau)))/2 + 1/2)*exp(Delta/2)/4 - f*((sqrt(2)*exp(-(-sigma*sqrt(tau) + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))**2/2)/(2*sqrt(pi)*sigma*sqrt(tau)) - sqrt(2)*exp(-(-sigma*sqrt(tau) + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))**2/2)/(2*sqrt(pi)*sigma*sqrt(tau)))*exp(mu*tau) - sqrt(2)*exp(-(Delta - tau*(mu - sigma**2/2))**2/(2*sigma**2*tau))/(2*sqrt(pi)*sigma*sqrt(tau)) + sqrt(2)*exp(-(-Delta - tau*(mu - sigma**2/2))**2/(2*sigma**2*tau))/(2*sqrt(pi)*sigma*sqrt(tau)))/(2*(exp(Delta/2) - 1)) + f*((erf(sqrt(2)*(-sigma*sqrt(tau) + (-Delta - tau*(mu - si

In [37]:
diff(v_bar, l)

-theta*Integral((-erf(sqrt(2)*(-sigma*sqrt(t) + (-Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(t) + (Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2 + erf(sqrt(2)*(Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2, (t, 0, tau))/(l + 1 - exp(-Delta/2))**2 - ((1/2 - erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau) + (erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + 1/2)*exp(mu*tau))*(exp(Delta/2) + 1)**2*exp(tau*(mu + 3*sigma**2/4)/2)/4 - ((-erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(-3*sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(mu*tau) + (-erf(sqrt(2)*(sigma*sqrt(tau)/2 + (-Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2 + erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau

In [38]:
diff(v_bar, delta, 2)

f*((erf(sqrt(2)*(sigma*sqrt(tau) + (2*Delta + tau*(2*mu - sigma**2))/(2*sigma*sqrt(tau)))/2) - 1)*exp(mu*tau) + erf(sqrt(2)*(Delta - tau*(2*mu - sigma**2)/2)/(2*sigma*sqrt(tau))) - 1)*exp(Delta/2)/16 - f*((erf(sqrt(2)*(sigma*sqrt(tau) - (2*Delta - tau*(2*mu - sigma**2))/(2*sigma*sqrt(tau)))/2) + erf(sqrt(2)*(sigma*sqrt(tau) + (2*Delta + tau*(2*mu - sigma**2))/(2*sigma*sqrt(tau)))/2) - 2*erf(sqrt(2)*sqrt(tau)*(sigma - (-2*mu + sigma**2)/(2*sigma))/2))*exp(mu*tau) + erf(sqrt(2)*(Delta - tau*(2*mu - sigma**2)/2)/(2*sigma*sqrt(tau))) - erf(sqrt(2)*(Delta + tau*(2*mu - sigma**2)/2)/(2*sigma*sqrt(tau))) + 2*erf(sqrt(2)*sqrt(tau)*(mu - sigma**2/2)/(2*sigma)))*exp(Delta/2)/(16*(exp(Delta/2) - 1)**2) + f*((erf(sqrt(2)*(sigma*sqrt(tau) - (2*Delta - tau*(2*mu - sigma**2))/(2*sigma*sqrt(tau)))/2) + erf(sqrt(2)*(sigma*sqrt(tau) + (2*Delta + tau*(2*mu - sigma**2))/(2*sigma*sqrt(tau)))/2) - 2*erf(sqrt(2)*sqrt(tau)*(sigma - (-2*mu + sigma**2)/(2*sigma))/2))*exp(mu*tau) + erf(sqrt(2)*(Delta - tau*(2*mu

In [39]:
diff(v_bar, l, 2)

theta*Integral(-(erf(sqrt(2)*(sigma*sqrt(t) - (2*Delta - t*(2*mu - sigma**2))/(2*sigma*sqrt(t)))/2) - erf(sqrt(2)*(sigma*sqrt(t) + (2*Delta + t*(2*mu - sigma**2))/(2*sigma*sqrt(t)))/2))*exp(mu*tau) + erf(sqrt(2)*(Delta - t*(2*mu - sigma**2)/2)/(2*sigma*sqrt(t))) + erf(sqrt(2)*(Delta + t*(2*mu - sigma**2)/2)/(2*sigma*sqrt(t))), (t, 0, tau))/(l + 1 - exp(-Delta/2))**3

In [40]:
diff(diff(v_bar, delta), l)

-theta*Integral((sqrt(2)*exp(-(-sigma*sqrt(t) + (Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))**2/2)/(2*sqrt(pi)*sigma*sqrt(t)) + sqrt(2)*exp(-(-sigma*sqrt(t) + (-Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))**2/2)/(2*sqrt(pi)*sigma*sqrt(t)))*exp(mu*tau) + sqrt(2)*exp(-(Delta - t*(mu - sigma**2/2))**2/(2*sigma**2*t))/(2*sqrt(pi)*sigma*sqrt(t)) + sqrt(2)*exp(-(-Delta - t*(mu - sigma**2/2))**2/(2*sigma**2*t))/(2*sqrt(pi)*sigma*sqrt(t)), (t, 0, tau))/(l + 1 - exp(-Delta/2))**2 + theta*exp(-Delta/2)*Integral((-erf(sqrt(2)*(-sigma*sqrt(t) + (-Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2 + erf(sqrt(2)*(-sigma*sqrt(t) + (Delta - t*(mu - sigma**2/2))/(sigma*sqrt(t)))/2)/2)*exp(mu*tau) - erf(sqrt(2)*(-Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2 + erf(sqrt(2)*(Delta - t*(mu - sigma**2/2))/(2*sigma*sqrt(t)))/2, (t, 0, tau))/(l + 1 - exp(-Delta/2))**3 - ((1/2 - erf(sqrt(2)*(sigma*sqrt(tau)/2 + (Delta - tau*(mu - sigma**2/2))/(sigma*sqrt(tau)))/2)/2)*exp(-mu*tau) + (erf(sqrt(2)*(-3*sigm